In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt

from getData import *
from vggFace import vggFace

model = vggFace()


Using TensorFlow backend.


Constructing model
loading weights


weights loaded


In [5]:
# CLEAR TEST/TRAIN DATA
tr_imgs = 0
test_imgs = 0

In [2]:

tr_imgs, tr_labels, tr_ids = getTrainingDataReplay_w_id()
test_imgs, test_labels,test_ids = getTestDataReplay_w_id()


# tr_imgs, tr_labels = getTrainingDataOuluP1()
# test_imgs, test_labels = getTestDataOuluP1()

print(tr_imgs.shape, tr_labels.shape)
print(test_imgs.shape, test_labels.shape)

(2400, 224, 224, 3) (2400,)
(3200, 224, 224, 3) (3200,)


In [9]:
# test_imgs = test_imgs[0:20,:,:,:]
# print(test_imgs.shape)
print(tr_ids.shape)
dummy_test_ids = np.random.rand(3200,15)

(2400, 15)


In [ ]:
# test_imgs, test_labels = getTestDataOuluP1()
# test_imgs, test_labels = getTestDataReplay()
# print(test_imgs.shape, test_labels.shape)

In [3]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
# pad_model = Model(inputs=model.layers[0].input, outputs=model.get_layer('last_feature_layer').output)
x = GlobalAveragePooling2D()(model.get_layer('last_feature_layer').output)
x = Dense(100,activation='relu')(x)
x = Dense(80,activation='relu')(x)
x = Dense(1,activation='tanh', name='pad')(x)

y = Flatten()(model.get_layer("fr_classif5").output)
y = Dense(15)(y)
y = Activation('softmax', name='fr')(y)

pad_model = Model(inputs=model.layers[0].input, outputs=[x,y])


# for layer in pad_model.layers:
#     print(layer.name)

pad_model.get_layer("fr_classif1").trainable = False
pad_model.get_layer("fr_classif2").trainable = False
pad_model.get_layer("fr_classif3").trainable = False
pad_model.get_layer("fr_classif4").trainable = False
pad_model.get_layer("fr_classif5").trainable = False



pad_model.summary()

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])
losses = {'pad' : 'mse', 'fr' : 'categorical_crossentropy'}
# lossWeights = {"pad": 1.0, "fr": 1.0}
# loss_weights=lossWeights
pad_model.compile(optimizer='adam', loss=losses, metrics=['accuracy','categorical_accuracy'])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
zero_padding2d_1_input (InputLa (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 226, 226, 3)  0           zero_padding2d_1_input[0][0]     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 64) 1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 226, 226, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [39]:
history = pad_model.fit(tr_imgs, {"pad":tr_labels, "fr":tr_ids}, batch_size=16, epochs=2)
print(history.history)
# pad_model.save("padModel3_Replay_28122018.h5")

Epoch 1/2


  16/2400 [..............................] - ETA: 59s - loss: 1.1149 - pad_loss: 0.4836 - fr_loss: 0.6314 - pad_acc: 0.8750 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8125 - fr_categorical_accuracy: 0.8125

  32/2400 [..............................] - ETA: 56s - loss: 0.9013 - pad_loss: 0.3668 - fr_loss: 0.5345 - pad_acc: 0.9062 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8438 - fr_categorical_accuracy: 0.8438

  48/2400 [..............................] - ETA: 55s - loss: 0.8690 - pad_loss: 0.3859 - fr_loss: 0.4831 - pad_acc: 0.8750 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8750 - fr_categorical_accuracy: 0.8750

  64/2400 [..............................] - ETA: 54s - loss: 0.9224 - pad_loss: 0.3569 - fr_loss: 0.5655 - pad_acc: 0.8750 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8438 - fr_categorical_accuracy: 0.8438

...

2352/2400 [============================>.] - ETA: 1s - loss: 0.7146 - pad_loss: 0.1275 - fr_loss: 0.5871 - pad_acc: 0.9498 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8163 - fr_categorical_accuracy: 0.8163

2368/2400 [============================>.] - ETA: 0s - loss: 0.7138 - pad_loss: 0.1268 - fr_loss: 0.5870 - pad_acc: 0.9497 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8167 - fr_categorical_accuracy: 0.8167

2384/2400 [============================>.] - ETA: 0s - loss: 0.7127 - pad_loss: 0.1259 - fr_loss: 0.5868 - pad_acc: 0.9501 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8163 - fr_categorical_accuracy: 0.8163

2400/2400 [==============================] - 55s 23ms/step - loss: 0.7098 - pad_loss: 0.1251 - fr_loss: 0.5847 - pad_acc: 0.9504 - pad_categorical_accuracy: 1.0000 - fr_acc: 0.8167 - fr_categorical_accuracy: 0.8167


{'loss': [1.1426945899923642, 0.7098039455215136], 'pad_loss': [0.2502925269713855, 0.12510114946905465], 'fr_loss': [0.8924020623167356, 0.5847027956570188], 'pad_acc': [0.9225, 0.9504166666666667], 'pad_categorical_accuracy': [1.0, 1.0], 'fr_acc': [0.7604166666666666, 0.8166666666666667], 'fr_categorical_accuracy': [0.7604166666666666, 0.8166666666666667]}


In [40]:
pad_model.save("padModel3_Replay_28122018_13epc.h5")

In [56]:
from keras.models import load_model
# model = load_model("MyVGGface.h5")
pad_model = load_model("padModel3_Replay_28122018_13epc.h5")

In [32]:
score = pad_model.evaluate(test_imgs,{"pad":test_labels,"fr":dummy_test_ids},batch_size=16 )
print(score)

  16/3200 [..............................] - ETA: 33s

  32/3200 [..............................] - ETA: 31s

  48/3200 [..............................] - ETA: 31s

  64/3200 [..............................] - ETA: 29s

  80/3200 [..............................] - ETA: 29s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 31s 10ms/step


[44.24645148277283, 0.34830290879280645, 43.89814862251282, 0.8853125, 1.0, 0.06625, 0.06625]


In [57]:
scores = pad_model.predict(test_imgs,batch_size=16,verbose=1)
print(scores[1].shape)
scores_pad = scores[0]
features = scores[1]


  16/3200 [..............................] - ETA: 1:27

  32/3200 [..............................] - ETA: 57s 

  48/3200 [..............................] - ETA: 48s

  64/3200 [..............................] - ETA: 43s

  80/3200 [..............................] - ETA: 39s

...

3152/3200 [============================>.] - ETA: 0s

3168/3200 [============================>.] - ETA: 0s

3184/3200 [============================>.] - ETA: 0s

3200/3200 [==============================] - 31s 10ms/step


(3200, 15)


In [58]:
np.savetxt("scores/scores_model3_13epc_Replay_28122018.txt",scores_pad)
# np.savetxt("scores/features_model3_Replay_28122018.txt",features)
# np.savetxt("scores/truelabels_model3_Replay_28122018.txt",test_labels) #TODO maybe save PAD and FR scores separatly

In [ ]:
print(scores)

[[-0.99516386]
 [-0.7635757 ]
 [-0.99999994]
 ...
 [ 0.99604744]
 [ 0.9993456 ]
 [ 0.99867606]]


In [ ]:
from keras.utils import plot_model
plot_model(pad_model, to_file='model.png',show_shapes=True)